# 머신러닝을 이용한 번역서비스의 언어감지
---


## 1.연구목표 설정

- 유사서비스 : 카카오번역, 파파고, 구글번역..
- 언어 감지 하는 방법은 지도학습법을 사용하겟다
  - 알파벳 (26:피쳐)의 빈도 -> en, fr, id, t..
  - 독립변수의 빈도를 보고 레이블(정답,종속변수)를 판단하게 되는 형태 -> 분류(지도학습)
  - 비영어권 제외, 알파벳 사용 국가 분류 총데이터도 제공된 자료만으로 사용(4개국가)
  - 알고리즘의 업데이트 시점은 로그를 축적하여 1만건 누적되면 추가 학습후 갱신하는 방식으로 진행

|step|단계명|수행내용|
|:--:|--:|:--|
|1|연구목표 설정|- 언어를 감지하는 웹서비스이다<br>- 머신러닝의 지도학습법의 분류 알고리즘을 이용하여 분류기를 구현하겠다<br>- 최적화 부분은 생략<br>- 정확도에 대한 정량적인 목표는 생략|
|2|데이터 수집/확보|- 실전:위키피디아에서 텍스트를 웹스크래핑으로 수집하여 데이터를 확보<br>- 프로토타입구현:제공 데이터를 사용하겠다|
|3|데이터 준비|- 전처리(데이터 원재료에서 아파벳을 빼고 모두 제거->정규식)<br>- 빈도 계산, 정규화<br>- 훈련용데이터와 학습용데이터를 75:25로 나눈다 |
|4|데이터 분석|- 영어권 언어별로 알파벳의 출현(사용) 빈도가 다르다는 명제(논문,주장)을 근거를 증명<br>- 탐색적분석,  EDA를 이용하여 표현(막대차트, 선형차트)|
|5|데이터 모델링 구축|- 알고리즘 선택<br>- 학습<br>- 예측<br>- 성능평가<br>- 모델덤프|
|6|시스템 통합|- 웹서비스구축<br>- 모델덤프로드모듈구현<br>- 언어감지를 처리하는 모듈 구현<br>- 번역 요청 로그처리|

## 2. 데이터 수집/확보
- 실전:위키피디아에서 텍스트를 웹스크래핑으로 수집하여 데이터를 확보
- 프로토타입구현:제공 데이터를 사용

### 2.1 정규식
[정규식기초_이동](https://github.com/Jerrykim91/Bigdata_Analytics/blob/master/Python_Projects_with_poo/Python_Basic_ipynb/Python_Basic_P000_06.ipynb)

In [1]:
# 정규식 
import re

In [2]:
# 대상언어별로 준비 => 대량의 데이터를 확보 
# 충분한 학습을 수행하기 위한 재료 

---

## 3. 데이터 준비

####  제공된 데이터 사용 
- 파일을 한번에 다읽기위해서 => 특정폴더 밑에서 특정패턴을 가진 파일 지정(목록화)
    - 파일명 : 언어코드(나라코드)_넘버.txt


#### 구상하기
- 데이터를 읽고 데이터를분석 =>  알파벳의 빈도 계산 
- 언어코드도 획득해서, 해당언어(국가언어)에 알파벳의 빈도를 표현 
- 알파벳별 빈도(feature), 언어코드(label)
- train 데이터는 
    - rows가 20개(국가별 5개)
    - colums(26(알파벳)+1(정답))
- shape >>> (20,27)

---

### 의사결정 코드 
- 제공된 데이터를 사용해서 특정패턴을 가진 파일을 목록화
- 구조과 규칙이 변하지 않는다는 전제하 작업


### Step 01. 언어코드 획득

- 변수명 : text 
    1. 파일에서 파일명 획득 
        =>  파일을 목록화 
        - 내장함수 이용 => `import glob`
            - 파일명을 전부 획득(파일리스트 획득)

    2. text 파일 한개 확인  
        => file_list의 개수만큼 반복
        => 20번 반복(list의 갯수)
           
    3. 언어코드 획득 및 추출
         - 파일에서 파일명 획득 
             => 언어 코드존재 => `['./input/train\\'**en**'-1.txt']`
                 => 언어(국가)코드 : 'tr, en' 
         - 정규식을 이용해서 추출  
            => 언어코드 확득 `re.compile( '^[a-z]{2}' )`

---

### Step 02. 알파벳별 빈도수 획득 
           
1. 전처리 통해서 영어만 남김
    - 파일을 불러 옴
        - 읽는다 
            =>  `import os`
            => 읽을 때 *소문자만* 읽어 옴 
    - 알파벳만 남긴다 
        - 정규식 이용 `import re`
        
    - 알파벳 별로 할당할 공간 생성 
        - 하나하나 리스트로 담아도 되지만 `자동화를위해 ord()를 사용`    
        - 아스키코드로 문자를 숫자로 변환     
        - a->z 까지는 순서가 존재함으로( ASCII )    
        - 기준은 소문자 a   
        - 아래 그림 참조    
        
![아스키 코드](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile5.uf.tistory.com%2Fimage%2F216CE84C52694FF02054D4)


### Step 03. 알파벳별 빈도를 계산  
    - 텍스트 파일을 읽어서 => 알파벳만 남김 

- 데이터를 확인해보니 값의 편차가 큼      



- 데이터는 빈도를 표현함 
    => 최대최소의 의미보다 총 빈도에서 특정 알파벳이 얼마나 등장했는지 확인
    => 이를 기준으로 정규화 처리 진행 
    
    => 데이터 준비단계
        => 데이터의 품질을 향상을 목표를 가지고
        => 주 수단으로 정해야함 

### Step 04
 

In [3]:
# stp =list()
# for i in range(len(file_list)):
#     for j in file_list: #  순서만 
# #         print(j)
#         stp.append(j)

In [4]:
# 파일 리스트 획득
import glob

In [5]:
# 파일 목록 보기 =>  파일명 전부 확득  
path = './input/train/*.txt'
file_list = glob.glob(path)
file_list[:2], len(file_list)

(['./input/train\\en-1.txt', './input/train\\en-2.txt'], 20)

In [6]:
file_list.sort()

### Step 01. 언어코드 획득

- 변수명 : text 
    1. 파일에서 파일명 획득 
        =>  파일을 목록화 
        - 내장함수 이용 => `import glob`
            - 파일명을 전부 획득(파일리스트 획득)

    2. text 파일 한개 확인  
        => file_list의 개수만큼 반복
        => 20번 반복(list의 갯수)
           
    3. 언어코드 획득 및 추출
         - 파일에서 파일명 획득 
             => 언어 코드존재 => `['./input/train\\'**en**'-1.txt']`
                 => 언어(국가)코드 : 'tr, en' 
         - 정규식을 이용해서 추출  
            => 언어코드 확득 `re.compile( '^[a-z]{2}' )`

In [7]:
file_list[0]

'./input/train\\en-1.txt'

In [8]:
# 언어코드 획득 
# 파일에서 파일명 획득 => 언어 코드존재  => 'tr, en'
file_list[0].split('/')[-1][:2]
# 언어코드 추출 

'tr'

In [9]:
# 외장함수를 활용하여 처리 
import os 

In [10]:
file_list[0].split('/')[-1][6:]

'en-1.txt'

In [11]:
# 파일명 획득 
# file_list[0].split('/')[-1][6:]과 결과는 동일

# os.path.basename() 
# 입력받은 경로의 기본 이름(base name)을 반환
name = os.path.basename(file_list[0])
name

'en-1.txt'

In [12]:
# 정규식으로 언어코드획득
import re
p         = re.compile( '^[a-z]{2,}' )
lng_code = p.match( name ).group()
# 언어 코드 
lng_code

'en'

### Step 02. 알파벳별 빈도수 획득 
           
1. 전처리 통해서 영어만 남김
    - 파일을 불러 옴
        - 읽는다 
            =>  `import os`
            => 읽을 때 *소문자만* 읽어 옴 
    - 알파벳만 남긴다 
        - 정규식 이용 `import re`
        
    - 알파벳 별로 할당할 공간 생성 
        - 하나하나 리스트로 담아도 되지만 `자동화를위해 ord()를 사용`    
        - 아스키코드로 문자를 숫자로 변환     
        - a->z 까지는 순서가 존재함으로( ASCII )    
        - 기준은 소문자 a   
        - 아래 그림 참조    
        
![아스키 코드](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile5.uf.tistory.com%2Fimage%2F216CE84C52694FF02054D4)


In [13]:
# 파일을 읽는다 (i/o)
with open(file_list[0], encoding= 'utf-8' ) as f : 
    # f 할당
    # f를 읽어드린다 
    text = f.read().lower()
    
    # 여기서 알파벳만 남긴다
    # 어떻게 남기냐 
    p = re.compile('[^a-z]*')
    text = p.sub('', text) # 일치 값만 제거 
    print(text[:100])
    
    pass # 알파벳만 남김 
    

themainhenryfordmuseumbuildinghousessomeoftheclassroomsforthehenryfordacademyhenryfordacademyisthefi


In [14]:
# 알파벳 벽고 카운트를 담는 그릇 => 하나의 이름으로 26의 정보를 담는 그릇 
# 파이썬 => 리스트 사용 => 수정이 가능, 순서 존재해서 적합 
# a -> z 까지 순서 존재 => 아스키(ASCII)
counts = [0 for i in range(26)] 
counts

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [15]:
# 리스트로 => 변수 26개를 표현 
# 알파벳 별로 카운트를 담을 그릇을 만든다 
# [0] *26 => 동일 
counts = []

for i in range(26): 
    i = 0
    counts.append(i)
counts   

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [16]:
[0]*26

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [17]:
a = []
b = [0]*26
# extend 와 append의 차이는? 
a.extend('2')
a

['2']

In [18]:
# text에서 한글자씩 읽어서 -> a면 counts[0]+=1 -> counts[a-a]+=1
# z -> counts[25]+=1 -> counts[z-a]+=1 
# t -> counts[?]+=1 -> counts[t-a]+=1 
# ord() 
#    - 특정 문자의 아스키코드값을 리턴
# a-z까지는 연속적으로 아스키값이 배치
# 따라서 특정 문자의 위치는 ord('소문자')-ord('a')
ord('a'), ord('z'), (ord('z')-ord('a'))

(97, 122, 25)

In [19]:
counts = [0 for i in range(26)] 
# 연속적이라서 아스키 사용 
ASCII_A = ord('a')  # a를 기준 데이터 
# ch에 들어가는 아스키값과 기준데이터를 연산해 다른 알파벳을 알수있음

for i in text: 
    # print(ch)
    # 한개뽑인 문자의 아스키값 ord(i)
    counts[(ord(i)-ASCII_A)] += 1
    # break 
    
print(counts,'\n',len(counts))


[349, 59, 210, 212, 484, 72, 88, 201, 340, 8, 25, 247, 121, 356, 412, 76, 0, 357, 282, 370, 119, 45, 65, 3, 92, 2] 
 26


In [20]:
sum(counts), len(text)

(4595, 4595)

In [21]:
# 람다함수 
# 대체 : 파이썬 내장함수 map(), DataFrame의 apply()

# map('함수','데이터 덩어리')
# list(map( 함수 , 데이터 덩어리))

# 함수는 map =>  자동으로 멤버수만큼 호출
# x:맴버 한개 한개,...
def check_norm(x):
    # print(x)  # 함수 호출 값을 확인 
    # 알파벳별 빈도/총 빈도(= 총 알파벳 문자열 개수)
    return x / len(text)       # x / 총빈도 

# frequences = list(map( check_norm, counts))
# print(sum(frequences),frequences)

# 한줄로 줄일수 있음 => 람다를 이용 
# 단정 휘발성에 => 일회성 


# 람다식을 이용해서 함수 코드를 한줄로 변환 
# list(map( lambda x:x/len(text), counts))
# lambda 인자 : 함수를 표현한식, 데이터 덩어리 
# 수행문이 한줄일 경우 가능 
# 알파벳의 총 빈도수 
frequences = list(map( lambda x:x/len(text), counts))
frequences, len(frequences)

([0.07595212187159957,
  0.012840043525571273,
  0.04570184983677911,
  0.04613710554951034,
  0.10533188248095757,
  0.015669205658324265,
  0.019151251360174103,
  0.043743199129488576,
  0.07399347116430903,
  0.0017410228509249185,
  0.00544069640914037,
  0.05375408052230685,
  0.026332970620239392,
  0.07747551686615888,
  0.08966267682263329,
  0.016539717083786723,
  0.0,
  0.07769314472252448,
  0.061371055495103376,
  0.08052230685527748,
  0.02589771490750816,
  0.009793253536452665,
  0.014145810663764961,
  0.0006528835690968443,
  0.02002176278563656,
  0.0004352557127312296],
 26)

In [22]:
# 텍스트 파일 1개에 대한 최종 결과물
# lang_code   :  언어 코드 
# frequences  :  알파벳 별로 담긴 평균 빈도수  
#     => 알파벳별 빈도/총 빈도(= 총 알파벳 문자열 개수) 
# print( len(lang_code), len(frequences) )


# 텍스트 파일 1개(en)에 대한 최종 결과물
print( lng_code, frequences )

en [0.07595212187159957, 0.012840043525571273, 0.04570184983677911, 0.04613710554951034, 0.10533188248095757, 0.015669205658324265, 0.019151251360174103, 0.043743199129488576, 0.07399347116430903, 0.0017410228509249185, 0.00544069640914037, 0.05375408052230685, 0.026332970620239392, 0.07747551686615888, 0.08966267682263329, 0.016539717083786723, 0.0, 0.07769314472252448, 0.061371055495103376, 0.08052230685527748, 0.02589771490750816, 0.009793253536452665, 0.014145810663764961, 0.0006528835690968443, 0.02002176278563656, 0.0004352557127312296]


### 위의 구성을 함수화 시킴 

In [23]:
import os, re

In [24]:
base_name = os.path.basename(file_list[0])
# 1.1 정규식 
# => 오류 =>  p = re.compile('^[a-z{2}]')
p         = re.compile('^[a-z]{2}')
lng_code  = p.match(base_name).group()

lng_code

'en'

In [25]:
(ord('b') - ord('a'))+1

# i += 1

2

In [26]:
counts = [0 for n in range(26)]
ASCII_A = ord('a')

for i in text:
#         print(i)
# ord('a') - ord('a')  = 0 
    # 각 알파벳 센다 
    counts[(ord(i)-ASCII_A)] += 1   
counts 
# 1. [0]*26
# 2. i += 1  
# 3. (ord('b') - ord('a'))+1

[349,
 59,
 210,
 212,
 484,
 72,
 88,
 201,
 340,
 8,
 25,
 247,
 121,
 356,
 412,
 76,
 0,
 357,
 282,
 370,
 119,
 45,
 65,
 3,
 92,
 2]

In [27]:
# 인자 : file_path(./input/train\\en-2.txt') =>  덩어리중 한개의 경로를 담을 예정 

def get_data(file_path):
    # 1. 언어코드 획득 
    # 입력받은 경로의 기본 이름(base name)을 반환  =>  'en-2.txt'
    base_name = os.path.basename(file_path)
    # 1.1 정규식 
    p         = re.compile('^[a-z]{2}') # 알파벳만 출력하는데 [a-z]을 {2}개 출력
    lng_code  = p.match(base_name).group()

    # 단어의 빈도 
    try : 
        with open(file_path,  encoding= 'utf-8') as f : 
            text  = f.read().lower()
            p     = re.compile('[^a-z]*')
            text  = p.sub( '' ,  text )
        print('정상작동', f,'\n','='*70)

    except Exception as e : 
        print('에러발생', e)
    
    # 알파벳 그릇 만들기 
    counts = [0 for n in range(26)]
    ASCII_A = ord('a')
    for i in text:
        counts[(ord(i)-ASCII_A)] += 1   # 3. (ord('b') - ord('a'))+1
        
    total_counts = sum(counts)  
    frequences = list(map(lambda x:x/total_counts, counts)) # 알파벳 갯수 
    
    # 결과값 리턴
    return (lng_code, frequences)


In [28]:
get_data( file_list[2] )

정상작동 <_io.TextIOWrapper name='./input/train\\en-3.txt' mode='r' encoding='utf-8'> 


('en',
 [0.07164591977869986,
  0.012171507607192255,
  0.04564315352697095,
  0.0326417704011065,
  0.12005532503457815,
  0.014661134163208852,
  0.02517289073305671,
  0.02351313969571231,
  0.0946058091286307,
  0.0024896265560165973,
  0.0016597510373443983,
  0.05698478561549101,
  0.022406639004149378,
  0.07551867219917012,
  0.07662517289073306,
  0.02074688796680498,
  0.0016597510373443983,
  0.05394190871369295,
  0.08796680497925312,
  0.08105117565698479,
  0.029045643153526972,
  0.018810511756569847,
  0.01189488243430152,
  0.0005532503457814661,
  0.017980636237897647,
  0.0005532503457814661])

In [29]:
# 훈련용 데이터, 테스트용 데이터 로드 
# 중간경로가 train 혹은 test 

# path = 'train'
def load_data(path = 'train'):
    
    # 파일 경로 => format으로 담는다 
    file_lists = glob.glob('./input/{}/*txt'.format(path))
    
    # 리스트 : 정답을 할당
    labels = list()
    freqs  = list()
    
    # 하나씩 출력 
    for f in file_lists:
        # 파일 1개당, 정답, 빈도수리스트 리턴 
        lng,freq = get_data(f)
        labels.append( lng )
        freqs.append( freq )
        
    return {'labels':labels, 'freqs': freqs } #  {'labels':['en','fr'], 'freqs': [[],[]] } 


In [40]:
train_data = load_data()
test_data  = load_data('test')

정상작동 <_io.TextIOWrapper name='./input/train\\en-1.txt' mode='r' encoding='utf-8'> 
정상작동 <_io.TextIOWrapper name='./input/train\\en-2.txt' mode='r' encoding='utf-8'> 
정상작동 <_io.TextIOWrapper name='./input/train\\en-3.txt' mode='r' encoding='utf-8'> 
정상작동 <_io.TextIOWrapper name='./input/train\\en-4.txt' mode='r' encoding='utf-8'> 
정상작동 <_io.TextIOWrapper name='./input/train\\en-5.txt' mode='r' encoding='utf-8'> 
정상작동 <_io.TextIOWrapper name='./input/train\\fr-10.txt' mode='r' encoding='utf-8'> 
정상작동 <_io.TextIOWrapper name='./input/train\\fr-6.txt' mode='r' encoding='utf-8'> 
정상작동 <_io.TextIOWrapper name='./input/train\\fr-7.txt' mode='r' encoding='utf-8'> 
정상작동 <_io.TextIOWrapper name='./input/train\\fr-8.txt' mode='r' encoding='utf-8'> 
정상작동 <_io.TextIOWrapper name='./input/train\\fr-9.txt' mode='r' encoding='utf-8'> 
정상작동 <_io.TextIOWrapper name='./input/train\\id-11.txt' mode='r' encoding='utf-8'> 
정상작동 <_io.TextIOWrapper name='./input/train\\id-12.txt' mode='r' encoding='utf-8'> 
정

### 현재파일을 저장 

In [31]:
# 덤프 침 => 현재 단계 결과물을 저장 
import json

In [41]:
# json 형태의 파일로 저장 

file_path = './input/'
full_path = file_path + 'labels_freqs_data'

try: 
    with open('{}.json'.format(full_path),'w', encoding='utf-8') as f: 
        # json 구조로 저장  => [train_data, test_data], f
        json.dump([train_data, test_data],f)
        print('정상동작')
except Exception as e:
    print('에러발생', e)

정상동작


---

# 4. 데이터 분석
 - 알파벳의 빈도가 영어권 국가별로 다름

### 1. 분석용 패키지 업로드

In [33]:
# 1. 분석용 패키지 업로드 
import pandas as pd 
import numpy as np

# 그래프 패키지 
import matplotlib.pyplot as plt 
%matplotlib inline

In [34]:
# 변수
file_path = './input/'
file_name = 'labels_freqs_data'
full_path = file_path + file_name

### 2. 데이터 로드 

In [35]:
# import
import os, re , glob, json

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
%matplotlib inline

from string import ascii_lowercase

In [38]:
def try_load( name, option, encoding='utf-8'):
    #  문자열 안나오게 하려했는데 => 변수로 받아들여서 실패 
    # 그외엔 다 정상동작 
    
    file_path = './input/' + '{}.json'.format(name)
    
    try:
        with open( file_path , option) as f:
            if option == 'w' :
                input_data = input('덤프시킬 데이터를 입력하세요')
#                 print(input_data)
                json.dump(input_data,f)
                

            elif option == 'r' :  
                tmp = json.load(f)
                print(tmp)
                print('\n길이 =', len(tmp)) 
                return tmp

        print('정상동작')

    except Exception as e:
        print('에러발생', e )

    return 

In [39]:
try_load('test_data', 'r')

에러발생 Expecting value: line 1 column 1 (char 0)


In [97]:
try_load('test_data', 'w')


덤프시킬 데이터를 입력하세요


KeyboardInterrupt: 

In [45]:
# 위에서 생성한 json 파일을 불러옴
try:
    with open('{}.json'.format(full_path),'r', encoding='utf-8') as f:
        # json 불러오기
        tmp = json.load(f)
    print('정상출력','\n길이 =', len(tmp),tmp) # 0:train, 1:test
    # 길이가 2면 정상
    
except Exception as e :
    print('에러발생', e)

정상출력 
길이 = 2 [{'labels': ['en', 'en', 'en', 'en', 'en', 'fr', 'fr', 'fr', 'fr', 'fr', 'id', 'id', 'id', 'id', 'id', 'tl', 'tl', 'tl', 'tl', 'tl'], 'freqs': [[0.07595212187159957, 0.012840043525571273, 0.04570184983677911, 0.04613710554951034, 0.10533188248095757, 0.015669205658324265, 0.019151251360174103, 0.043743199129488576, 0.07399347116430903, 0.0017410228509249185, 0.00544069640914037, 0.05375408052230685, 0.026332970620239392, 0.07747551686615888, 0.08966267682263329, 0.016539717083786723, 0.0, 0.07769314472252448, 0.061371055495103376, 0.08052230685527748, 0.02589771490750816, 0.009793253536452665, 0.014145810663764961, 0.0006528835690968443, 0.02002176278563656, 0.0004352557127312296], [0.08417789436031954, 0.019911768212710148, 0.030404196971503518, 0.038869679265529984, 0.13699773458924527, 0.017407893167998092, 0.031238821986407535, 0.02742339334684631, 0.07535471563133421, 0.0026231071896983425, 0.009777035888875641, 0.042327411470132345, 0.024204125432216526, 0.0535352330

In [46]:
# 훈련 데이터 크기확인 => 20 개
len(tmp[0]['labels']),len(tmp[0]['freqs'])

(20, 20)

In [47]:
# 테스트 데이터 크기확인  => 8 개 
len(tmp[1]['labels']),len(tmp[1]['freqs'])

(8, 8)

In [48]:
# 훈련(train)데이터의 빈도데이터만 로드 
# df_freqs 변수에 데이터 프레임 형태의 tmp[0]['freqs'] 데이터를 할당
df_freqs = pd.DataFrame(tmp[0]['freqs']) # 0 => train_data
# 데이터 확인 
df_freqs.head(3) # 값이다름

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.075952,0.012840,0.045702,0.046137,0.105332,0.015669,0.019151,0.043743,0.073993,0.001741,...,0.000000,0.077693,0.061371,0.080522,0.025898,0.009793,0.014146,0.000653,0.020022,0.000435
1,0.084178,0.019912,0.030404,0.038870,0.136998,0.017408,0.031239,0.027423,0.075355,0.002623,...,0.005485,0.090140,0.071659,0.077739,0.030643,0.013712,0.013950,0.002027,0.010731,0.000596
2,0.071646,0.012172,0.045643,0.032642,0.120055,0.014661,0.025173,0.023513,0.094606,0.002490,...,0.001660,0.053942,0.087967,0.081051,0.029046,0.018811,0.011895,0.000553,0.017981,0.000553


In [49]:
df_freqs.shape

(20, 26)

In [50]:
# 훈련 데이터의 정답 데이터만 로드
# df_labels 변수에 데이터 프레임 형태의 tmp[0]['labels'] 데이터를 할당
df_labels = pd.DataFrame(tmp[0]['labels']) # 0 => train_data
# 데이터 확인 
df_labels.head(3) # 값이다름

,0
0,en
1,en
2,en


In [51]:
df_labels.shape

(20, 1)

In [52]:
#컬럼명을 변경 0 => label로 
df_labels.columns = ['label']

In [53]:
# 변경값을 확인 
df_labels.head(2)

,label
0,en
1,en


###  짚고 넘어가기 
#### `from string import ascii_lowercase`
- 보통사람들은 a-z까지 소문자를 가지고 오려고 할때 'abcdefg ....'와 같이 손수 알파벳을 입력
- 파이썬 => 데이터 상수로 정의 
- 다음부터는 이 패키지를 이용해서 파이썬 다운 명령어를 사용해보자     

```py
import string 

string.ascii_lowercase # 소문자 abcdefghijklmnopqrstuvwxyz
string.ascii_uppercase # 대문자 ABCDEFGHIJKLMNOPQRSTUVWXYZ
string.ascii_letters #대소문자 모두 
# abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
string.digits # 숫자 0123456789

```
---


#### df_freqs 컬럼이름을 a부터 z까지 변경 

In [54]:
from string import ascii_lowercase
# 소문자만 
ascii_lowercase


'abcdefghijklmnopqrstuvwxyz'

In [55]:
# 어떻게 동일한 값이니까 리스트에 담아서 집어 넣으면 
list(ascii_lowercase) # 이제 데이터를 집어 넣을수 있는 상태로 만들어 진걸 할당해보겠음 

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [56]:
# len(ascii_lowercase)
df_freqs.columns,len(list(ascii_lowercase))

(RangeIndex(start=0, stop=26, step=1), 26)

In [57]:
# df_freqs.columns.ascii_lowercase
#AttributeError: 'RangeIndex' object has no attribute 'ascii_lowercase'

In [58]:
# 컬럼명 변경
df_freqs.columns = list(ascii_lowercase)
df_freqs.head(2)

,a,b,c,d,e,f,g,h,i,j,...,q,r,s,t,u,v,w,x,y,z
0,0.075952,0.012840,0.045702,0.046137,0.105332,0.015669,0.019151,0.043743,0.073993,0.001741,...,0.000000,0.077693,0.061371,0.080522,0.025898,0.009793,0.014146,0.000653,0.020022,0.000435
1,0.084178,0.019912,0.030404,0.038870,0.136998,0.017408,0.031239,0.027423,0.075355,0.002623,...,0.005485,0.090140,0.071659,0.077739,0.030643,0.013712,0.013950,0.002027,0.010731,0.000596


#### 이제 빈도수 데이터와 정답 레이블을 결합 
- df_freqs, df_labels 결합 => (20, 27)
    - merge
    - concat

---

#### 짚고넘어가기 
서로다른 데이터프레임을 합치는 방법은 3가지가 있음

- merge(병합)
    - 두 데이터프레임을 각 데이터에 존재하는 고유값(key)을 기준으로 병합할때 사용
    - 공통된 하나의 열(or 행)을 기준으로 동일한 값을 가지는 행을 각 데이터프레임에서 찾은 후 모든 행을 가지도록 합치는 경우 
    
- concat(연결) 
    - 동일한 인덱스나 컬럼을 가진경을 연속적으로 붙이기 
    - 데이터프레임을 말그대로 물리적으로 이어 붙여주는 함수
         
- join(결합)
    - merge()함수를 기반으로 만들어졌기 때문에 기본 작동방식이 비슷
    - 행(->) 인덱스를 기준으로 결합한다는 점에서 차이

---

## Step 04. 데이터 모델링 구축

    - 알고리즘 선택
    - 지도학습의 분류 
        - SVM > SVC
    - 학습
        - 데이터(훈련용, 테스트용)
        - 통계적 모델링 : 훈련용, 테스트용 
        - 머신러닝 모델링: (훈련용, 검증용), 테스트용 
    - 예측
        - test 데이터를 사용
    - 성능평가
        - 분류 > 평가지표 > 정확도 정도 체크 
    - 모델덤프
        - 시슨템 통합단계 신규서비스에 제공하는 알고리즘 파일(학습된 모델이 적제된 ) 제공 
    
---
![선택시트](https://t1.daumcdn.net/cfile/tistory/9950844F5C986AD827)


### Step 04-1. 알고리즘 생성 

In [59]:
from sklearn import svm, metrics

In [60]:
# 분류기 이름 clf로 많이 사용 
clf = svm.SVC( gamma = 'auto') 
clf

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

---

### Step 04-2. 지도학습 분류 
- 훈련데이터 테스트데이터 
- 이미 위에서 생성완료 

In [65]:
freq = try_load('test_data', 'r')
# 버그 - 아니지 함수 오류 


길이 = 23


In [68]:
len(freq )

23

In [87]:
file_path_output = './output/'
file_name = 'labels_freqs_data'
full_path = file_path_output + file_name

In [88]:
# 위에서 생성한 json 파일을 불러옴
try:
    with open('{}.json'.format(full_path),'r', encoding='utf-8') as f:
        # json 불러오기
        freq = json.load(f)
    print('정상출력','\n길이 =', len(freq),freq) # 0:train, 1:test
    # 길이가 2면 정상
    
except Exception as e :
    print('에러발생', e)

에러발생 [Errno 2] No such file or directory: './output/labels_freqs_data.json'


In [71]:
len(freq)

2

In [72]:
# 훈련용 데이터
len(freq[0]['labels']), len(freq[0]['freqs'])

(20, 20)

In [62]:
#  데이터 확인겸 길이 확인 
len(train_data['freqs']),len(train_data['labels'])

(20, 20)

In [73]:
# 테스트용 데이터
len(freq[1]['labels']), len(freq[1]['freqs'])

(8, 8)

---

### Step 04-3. 학습(훈련)

In [63]:
# 훈련 
clf.fit(train_data['freqs'], train_data['labels'])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [74]:
clf.fit( freq[0]['freqs'], freq[0]['labels'] )

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

---

### Step 04-4. 예측
    - 한번도 접하지 못한 데이터를 이용하여 예측 수행 

In [75]:
# 예측에 사용된 데이터는 학습에 사용된 데이터와 
# 동일 형태를 취해야 한다
predict = clf.predict(freq[1]['freqs'])

In [76]:
predict

array(['en', 'en', 'fr', 'fr', 'en', 'id', 'tl', 'tl'], dtype='<U2')

---

### Step 04-5. 성능 평가 

- 정확도만 확인
- 목표한 수치에 도착할때까지 이전과정을 반복한다
    - 데이터가 작으면, 더 많이 확보
    - 데이터 준비 과정에서 문제가 있다면 보완
    - 알고리즘이 적절하지 않다면 교차 검증후 확인
    - 하이퍼파라미터 튜닝을 통해서 검증
    - 목표한 수치에 도착할때까지 반복(에자일 방식)

In [77]:
# 정확도 
metrics.accuracy_score( freq[1]['labels'],predict )

0.875

In [78]:
# 이쁘게 출력하기위해 프린트 사용 
print( metrics.classification_report(freq[1]['labels'], predict ))

             precision    recall  f1-score   support

         en       0.67      1.00      0.80         2
         fr       1.00      1.00      1.00         2
         id       1.00      0.50      0.67         2
         tl       1.00      1.00      1.00         2

avg / total       0.92      0.88      0.87         8



---

### Step 04-6. 모델 덤프 

- 6단계 시스템 통합에서 사용할수 있게 제공
- 모델을 향후 어떻게 갱신할 것인가? 이 주제는 생략
    - 서비스가 진행 => 데이터가 쌓인다 -> 다시 학습이 가능 -> 모델을 업그레이드 가능 -> 실시간갱신, 주기적 갱신 등등 -> 학습 방식에 대한 고찰(배치학습, 온라인 학습등등..)

In [79]:
# 예측 모델 저장 
from sklearn.externals import joblib

In [82]:
# 확장자는 자유롭게, 이름도 자유롭게, 시간정보는 입력
'''
save_path = '/content/drive/My Drive/clf_modle_202003101113.model'
joblib.dump( clf, save_path)
'''
joblib.dump(clf, './output/clf_lang_20200310.model')

['./output/clf_lang_20200310.model']

In [ ]:
joblib.dump(clf, '../output/clf_lang_20200310.model')

# 레이블!! (정답의 후보들)
with open('../data/clf_lang_labels.txt', 'w') as f:
    for label in list(set(train_data['labels'])):
          f.writelines( label+'\n' )

In [85]:
# 정답 저장 
label_dic = {
    'en':'영어',
    'fr':'프랑스어',
    'tl':'타갈리아어',
    'id':'인도네시아어'
}
label_dic

{'en': '영어', 'fr': '프랑스어', 'tl': '타갈리아어', 'id': '인도네시아어'}

In [86]:
# json으로 저장 
with open('clf_labels.json','w', encoding= 'utf-8') as f: 
    json.dump(label_dic, f)

---

## Step 05. 시스템 통합 




### 기본 Step
---
        
### Step_01
- 사용자가 번역에 필요한 글자를 입력 (알파벳만 사용, 영어권만 해당)  

### Step_02
- 언어감지라는 버튼을 클릭한다

### Step_03
- 언어를 읽어서 서버로 전송

### Step_04
- 서버에서 데이터를 알고리즘이 예측할수 있는 형태로 변환처리

### Step_05
- 예측

### Step_06
- 예측결과를 응답

### Step_07
- 응답 결과를 화면에 표시

---


In [ ]:
from IPython.display import Image

path = './input/6.시스템통합_최종산출물.png'
Image(path, width=500 )

# 최종 산출물  
# 웹기반 => 플라스크(간단하게 구현)
# 머신러닝(SVM,SVC)로 언어감지, 딥러닝(RNN, many to many 신경망)으로 언어 번역 

In [ ]:
# https://colab.research.google.com/drive/1j2x9zXnQ_Y4ZuJE4hwG0DhESFEpfrBkL#scrollTo=iJDRUPRyDeTV


## 구동원리

- 1. 사용자가 번역에 필요한 글자를 입력한다
  (알파벳만 사용, 영어권만 해당)  
- 2. 언어감지라는 버튼을 클릭한다
- 3. 언어를 읽어서 서버로 전송
- 4. 서버에서 데이터를 알고리즘이 예측할수 있는 형태로 변환처리
- 5. 예측
- 6. 예측결과를 응답
- 8. 응답 결과를 화면에 표시